In [1]:
%matplotlib widget

In [2]:
# curve.py  ── fixed single-figure widget version ─────────────────────
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets                     # ← add this line
from IPython.display import display, clear_output

PWM_MIN, PWM_MAX = 1000, 2000
TABLE_LEN        = 12

# ─── math helpers (unchanged) ────────────────────────────────────────
def _quad_bezier_y(x, p0x, p1x, p2x, p0y, p1y, p2y):
    a, b, c = p0x - 2*p1x + p2x, 2*p1x - 2*p0x, p0x - x
    if abs(a) < 1e-6:
        t = -c/b if abs(b) > 1e-6 else 0.0
    else:
        disc = b*b - 4*a*c
        if disc >= 0:
            root = disc**0.5
            t1, t2 = (-b + root)/(2*a), (-b - root)/(2*a)
            t = t1 if 0 <= t1 <= 1 else t2
        else:
            t = 0.0
    t = max(0.0, min(1.0, t))
    return (1-t)**2*p0y + 2*(1-t)*t*p1y + t**2*p2y

def _segment_y(x, thrMid, expo, thrHover):
    cp1x, cp1y = thrMid*(1-expo), thrHover
    cp2x, cp2y = thrMid + (1-thrMid)*expo, thrHover
    if x <= thrMid:
        return _quad_bezier_y(x, 0, cp1x, thrMid, 0, cp1y, thrHover)
    return  _quad_bezier_y(x, thrMid, cp2x, 1, thrHover, cp2y, 1)

def throttle_curve(thrMid=0.5, expo=0.0, thrHover=0.5, points=1000):
    xs = np.linspace(0, 1, points)
    ys = np.array([_segment_y(x, thrMid, expo, thrHover) for x in xs])
    return xs, ys

def lookup_table(thrMid=0.5, expo=0.0, thrHover=0.5,
                 length=TABLE_LEN, pwm_min=PWM_MIN, pwm_max=PWM_MAX):
    _, y = throttle_curve(thrMid, expo, thrHover, length)
    return np.rint(pwm_min + y*(pwm_max-pwm_min)).astype(int)

# ─── interactive UI ──────────────────────────────────────────────────
def interactive_curve():
    # ── 1 ▸ build figure while auto-display is OFF ────────────────
    fig_box = widgets.Output()             # figure goes here
    with fig_box, plt.ioff():
        fig, ax = plt.subplots(figsize=(6, 4))
        display(fig.canvas)                       # single, explicit display
    line,   = ax.plot([], [], lw=2, label="throttle curve")
    anchors = ax.scatter([], [], s=50, label="anchor pts")
    ctrls   = ax.scatter([], [], marker='x', s=80, label="control pts")
    seg1,   = ax.plot([], [], ls='--', lw=1, alpha=.6)
    seg2,   = ax.plot([], [], ls='--', lw=1, alpha=.6)
    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
    ax.set_xlabel('normalised stick'); ax.set_ylabel('normalised output')
    ax.grid(True); ax.legend(loc='upper left')

    # ── 2 ▸ widgets ------------------------------------------------

    text_box = widgets.Output()            # lookup-table print area

    # sliders
    s_mid   = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Mid')
    s_expo  = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.01, description='Expo')
    s_hover = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Hover')
    s_len   = widgets.IntSlider  (value=TABLE_LEN, min=4, max=32,     description='Tbl-len')
    sliders = widgets.VBox([s_mid, s_expo, s_hover, s_len])

    display(widgets.VBox([fig_box, text_box, sliders]))

    # ── 3 ▸ refresh logic (unchanged apart from text_box target) ---
    def refresh(_=None):
        thrMid, expo, thrHover, tlen = s_mid.value, s_expo.value, s_hover.value, s_len.value
        xs, ys = throttle_curve(thrMid, expo, thrHover)
        line.set_data(xs, ys)

        cp1x, cp1y = thrMid*(1-expo), thrHover
        cp2x, cp2y = thrMid + (1-thrMid)*expo, thrHover
        anchors.set_offsets(np.c_[[0, thrMid, 1], [0, thrHover, 1]])
        ctrls  .set_offsets(np.c_[[cp1x, cp2x],   [cp1y, cp2y]])
        seg1.set_data([0, cp1x, thrMid], [0, cp1y, thrHover])
        seg2.set_data([thrMid, cp2x, 1], [thrHover, cp2y, 1])
        ax.set_title(f"Mid={thrMid:.2f}  Hover={thrHover:.2f}  Expo={expo:.2f}")

        with text_box:
            clear_output(wait=True)
            print(f"lookupThrottleRC[{tlen}] = {lookup_table(thrMid, expo, thrHover, tlen).tolist()}")

        fig.canvas.draw_idle()

    for s in (s_mid, s_expo, s_hover, s_len):
        s.observe(refresh, 'value')
    refresh()                               # initial draw

In [3]:
interactive_curve()